# Magister en Ciencia de Datos - UDD
## DBAnalytics (Ciencia de Datos aplicada)
**Sprint 2: Datos transformados II**

En base a las funciones RFM del Sprint 1, crear nuevos features para cada variable en el dataset user_logs.csv y transactions.csv (sólo si aplica).
Crear nuevas funciones para construir features tales como:
Tendencias de actividad del msno entre diferentes rangos de tiempo usando diferentes variables, usando variables del dataset user_logs.csv
Métricas de la actividad del msno usando diferentes variables del dataset transactions.csv 
En base a las funciones implementadas (y otras definidas por cada grupo), se deben implementar al menos 300 features. 


In [5]:
import pandas as pd
import numpy as np
import gc

In [6]:
gc.enable()

* Lectura de archivos desde repositorio y contatenacion con nuevas versiones

In [7]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs://kk_data_udd/user_label_201703.csv#1563681061521361
gs://kk_data_udd/user_latent_satisfaction.csv#1

In [ ]:
df_train = pd.read_csv('gs://kk_data_udd/train.csv')
df_train = pd.concat((df_train, pd.read_csv('gs://kk_data_udd/train_v2.csv')), axis=0, ignore_index=True).reset_index(drop=True)

In [ ]:
df_test = pd.read_csv('gs://kk_data_udd/sample_submission_v2.csv')

In [ ]:
df_members = pd.read_csv('gs://kk_data_udd/members_v3.csv')

In [ ]:
df_transactions = pd.read_csv('gs://kk_data_udd/transactions.csv')
df_transactions = pd.concat((df_transactions, pd.read_csv('gs://kk_data_udd/transactions_v2.csv')), axis=0, ignore_index=True).reset_index(drop=True)
df_transactions = df_transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
df_transactions = df_transactions.drop_duplicates(subset=['msno'], keep='first')

In [ ]:
df_transactions.head()

* Feature Engineering

In [ ]:
df_transactions['discount'] = df_transactions['plan_list_price'] - df_transactions['actual_amount_paid']
df_transactions['is_discount'] = df_transactions.discount.apply(lambda x: 1 if x > 0 else 0)
df_transactions['membership_days'] = pd.to_datetime(df_transactions['membership_expire_date']).subtract(pd.to_datetime(df_transactions['transaction_date'])).dt.days.astype(np.int16)
df_transactions['amt_per_day'] = df_transactions['actual_amount_paid'] / df_transactions['payment_plan_days']

In [ ]:
df_transactions.head()

In [ ]:
df_train['marker'] = 1
df_test['marker'] = 0
df_combined = pd.concat([df_train, df_test], axis=0)

* Merging Members

In [ ]:
df_combined = pd.merge(df_combined, df_members, how='left', on='msno')
df_members = [];

In [ ]:
df_combined.head()

* Normalizamos genero

In [ ]:
gender = {'male':1, 'female':2}
df_combined['gender'] = df_combined['gender'].map(gender)

* Merging Transactions

In [ ]:
df_combined = pd.merge(df_combined, df_transactions, how='left', on='msno')
df_transactions =[]

In [ ]:
df_combined.head()

* Separamos entramiento y test, luego eliminamos el flag "marker"

In [ ]:
df_train = df_combined[df_combined['marker'] == 1]
df_test = df_combined[df_combined['marker']  == 0]

del df_train["marker"]
del df_test["marker"]

In [ ]:
del df_combined
gc.collect()
gc.enable()

* Extraemos los registros finales de log de usuario y los contatenamos con train y test

In [ ]:
!bq rm -f 'kk-churn:DATASET.user_log_features'

In [ ]:
%%bigquery 
create table `kk-churn.DATASET.user_log_features` as
select msno, count(msno) as msno_nro, max(date) as date_last, 
sum(num_25) as num_25_sum, min(num_25) as num_25_min, max(num_25) as num_25_max, avg(num_25) as num_25_avg, stddev(num_25) as num_25_std,
sum(num_50) as num_50_sum, min(num_50) as num_50_min, max(num_50) as num_50_max, avg(num_50) as num_50_avg, stddev(num_50) as num_50_std,
sum(num_75) as num_75_sum, min(num_75) as num_75_min, max(num_75) as num_75_max, avg(num_75) as num_75_avg, stddev(num_75) as num_75_std,
sum(num_985) as num_98_sum, min(num_985) as num_98_min, max(num_985) as num_98_max, avg(num_985) as num_98_avg, stddev(num_985) as num_98_std,
sum(num_100) as num_100_sum, min(num_100) as num_100_min, max(num_100) as num_100_max, avg(num_100) as num_100_avg, stddev(num_100) as num_100_std,
sum(num_unq) as num_unq_sum, min(num_unq) as num_unq_min, max(num_unq) as num_unq_max, avg(num_unq) as num_unq_avg, stddev(num_unq) as num_unq_std,
sum(total_secs) as total_secs_sum, min(total_secs) as total_secs_min, max(total_secs) as total_secs_max, avg(total_secs) as total_secs_avg, stddev(total_secs) as total_secs_std
from `kk-churn.DATASET.user_logs` group by msno 

In [ ]:
%%bigquery
select count(*) from `kk-churn.DATASET.user_log_features`

In [ ]:
import pandas as pd
query = """ 
select * from `kk-churn.DATASET.user_log_features`
        """
df = pd.read_gbq(query,project_id='kk-churn',dialect='standard')

In [ ]:
df.head()

In [ ]:
df_train = pd.merge(df_train, df, how='left', on='msno')

In [ ]:
df_test = pd.merge(df_test, df, how='left', on='msno')

In [ ]:
del df

In [ ]:
gc.collect()

In [ ]:
print(df_train.shape, df_test.shape)

In [ ]:
df_train.columns

In [ ]:
df_train = df_train.rename(columns = {'date_last':'user_log_date'})
df_test = df_test.rename(columns = {'date_last':'user_log_date'})

In [ ]:
def fix_time(df, time_cols):
    for time_col in time_cols:
        df[time_col] = pd.to_datetime(df[time_col], errors = 'coerce', format = '%Y%m%d')
    return df

In [ ]:
df_train = fix_time(df_train, time_cols = ['transaction_date', 'membership_expire_date', 'registration_init_time', 'user_log_date'])

In [ ]:
df_train[['transaction_date', 'membership_expire_date', 'registration_init_time', 'user_log_date']].head()

In [ ]:
df_test = fix_time(df_test, time_cols = ['transaction_date', 'membership_expire_date', 'registration_init_time', 'user_log_date'])

* Separamos las fechas en dias y meses

In [ ]:
date_dict = {'t_':'transaction_date', 'm_':'membership_expire_date', 'r_':'registration_init_time', 'l_':'user_log_date'}

In [ ]:
for m in date_dict:
    df_train[m+'month'] = [d.month for d in df_train[date_dict[m]]]
    df_train[m+'day'] = [d.day for d in df_train[date_dict[m]]]

In [ ]:
df_train['t_year'] = [d.year  for d in df_train['transaction_date']]
df_train['m_year'] = [d.year for d in df_train['membership_expire_date']]
df_train['r_year'] = [d.year for d in df_train['registration_init_time']]

In [ ]:
df_train['l_year'] = [d.year for d in df_train['user_log_date']]

In [ ]:
for m in date_dict:
        df_test[m+'month'] = [d.month for d in df_test[date_dict[m]]]
        df_test[m+'day'] = [d.day for d in df_test[date_dict[m]]]

In [ ]:
df_test['t_year'] = [d.year for d in df_test['transaction_date']]
df_test['m_year'] = [d.year for d in df_test['membership_expire_date']]
df_test['r_year'] = [d.year for d in df_test['registration_init_time']]
df_test['l_year'] = [d.year for d in df_test['user_log_date']]

In [ ]:
df_train['autorenew_&_not_cancel'] = ((df_train.is_auto_renew == 1) == (df_train.is_cancel == 0)).astype(np.int8)
df_test['autorenew_&_not_cancel'] = ((df_test.is_auto_renew == 1) == (df_test.is_cancel == 0)).astype(np.int8)
df_train['notAutorenew_&_cancel'] = ((df_train.is_auto_renew == 0) == (df_train.is_cancel == 1)).astype(np.int8)
df_test['notAutorenew_&_cancel'] = ((df_test.is_auto_renew == 0) == (df_test.is_cancel == 1)).astype(np.int8)

In [ ]:
list(df_train.columns)

* Guardamos archivos

In [ ]:
!gsutil ls -a gs://kk_data_udd

In [ ]:
df_test.to_csv('gs://kk_data_udd/test_sorted_v1.csv', index=False)

In [ ]:
df_train.to_csv('gs://kk_data_udd/train_sorted_v1.csv', index=False)

In [ ]:
df_user_logs = pd.read_csv('gs://kk_data_udd/user_logs.csv')
df_train = pd.read_csv('gs://kk_data_udd/train.csv')